<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [151]:
import numpy as np
import pydicom
import matplotlib.pyplot as plt
import math
import pandas as pd
from skimage.feature import greycomatrix
from skimage.feature import greycoprops
from skimage.measure import shannon_entropy
import pandas as pd
import os

from skimage.feature import greycomatrix as gray
from skimage.feature import greycoprops as props

In [152]:
#For this function, pass the original image without normalization
def calc_entropy(img):
    image_size = img.size
    values_count = []
    for i in range(np.min(img), np.max(img) + 1):
        occurrences = np.count_nonzero(img == i)
        values_count.append(occurrences)
    values_count = [i/image_size for i in values_count]
    s = 0
    for i in values_count:
        if i == 0:
            pass
        else:
            s = -1*i*math.log2(i) + s
    return s

In [153]:
#pass the image and the distance for glcm creation
def graycomatrix(img, distance):
    max_val = np.amax(img)
    min_val = np.amin(img)
    img = img + min_val*-1
    rows = img.shape[0]
    cols = img.shape[1]
    max_val = np.amax(img)
    min_val = np.amin(img)
    size_glcm = 0
    if min_val == 0:
        size_glcm = max_val + 1
    elif min_val < 0:
        size_glcm = max_val + 1 + min_val*-1
    else:
        size_glcm = max_val
    glcm = np.zeros((size_glcm, size_glcm))
    f = img.flatten()
    l = len(f)
    c = -1
    while(c < l-1):
        c += 1
        if c%cols >= cols-distance:
            pass
        else:
            glcm[f[c]][f[c + distance]]+= 1
    glcm = glcm/(np.apply_over_axes(np.sum, glcm, axes = (0, 1)))
    return glcm

In [154]:
def graycoprops(glcm):
    levels = glcm.shape[0]
    I, J = np.ogrid[0:levels, 0:levels]
    
    contrast_weights = (I - J) ** 2
    
    dissimilarity_weights = np.abs(I - J)
    
    homogeneity_weights = 1. / (1. + (I - J) ** 2)
    
    diff_i = I - np.apply_over_axes(np.sum, (I*glcm), axes = (0,1))[0, 0]
    diff_j = J - np.apply_over_axes(np.sum, (J*glcm), axes = (0,1))[0, 0]
    std_i = np.sqrt(np.apply_over_axes(np.sum, (glcm * (diff_i) ** 2),
                                           axes=(0, 1))[0, 0])
    std_j = np.sqrt(np.apply_over_axes(np.sum, (glcm * (diff_j) ** 2),
                                           axes=(0, 1))[0, 0])
    cov = np.apply_over_axes(np.sum, (glcm * (diff_i * diff_j)),
                                 axes=(0, 1))[0, 0]
    
    correlation = cov / (std_i * std_j)
    
    #Angular Second Moment
    asm = np.apply_over_axes(np.sum, (glcm ** 2), axes = (0, 1))[0, 0]
    
    #Energy
    energy = np.sqrt(asm)
    
    #Contrast
    contrast = np.apply_over_axes(np.sum, (glcm*contrast_weights), 
                                  axes = (0,1))[0][0]
    
    #Dissimilarity
    dissimilarity = np.apply_over_axes(np.sum, (glcm*dissimilarity_weights), 
                                  axes = (0,1))[0][0]
    
    #Homogeneity
    homogeneity = np.apply_over_axes(np.sum, (glcm*homogeneity_weights), 
                                  axes = (0,1))[0][0]
    
    return asm, energy, contrast, dissimilarity, homogeneity, correlation

In [155]:
dicom = pydicom.dcmread('/Users/rt/Desktop/College/Cancer Project/dicom/2/FATTU SHANKAR MAHALE/Z342')
img = dicom.pixel_array

In [156]:
glcm = graycomatrix(img, 1)

In [7]:
asm, energy, contrast, dissimilarity, homogeneity, correlation = graycoprops(glcm)

In [8]:
print(f'ASM = {asm} \nEnergy = {energy} \nContrast = {contrast} \nDissimilarity = {dissimilarity} \nHomogeneity = {homogeneity} \nCorrelation = {correlation}')

ASM = 0.044642321462772666 
Energy = 0.2112872960278792 
Contrast = 17434.552463001466 
Dissimilarity = 21.10976486056751 
Homogeneity = 0.3319213087125058 
Correlation = 0.9906727277194525
